In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
df_train = pd.read_csv('train.csv', index_col='id')
df_test = pd.read_csv('test.csv', index_col='id')

In [ ]:
import warnings

warnings.filterwarnings('ignore')

In [ ]:
df_train.head()

In [ ]:
df_train.isnull().sum()

In [ ]:
X_train = df_train.drop(df_train.columns[0], axis=1)
Y_train = df_train[df_train.columns[0]]
X_test = df_test.drop(df_test.columns[0], axis=1)
Y_test = df_test[df_test.columns[0]]


In [ ]:
Y_train[Y_train == 'e'] = 0

Y_train[Y_train == 'p'] = 1

Y_test[Y_test == 'e'] = 0

Y_test[Y_test == 'p'] = 1


In [ ]:
#multi hot encode:



In [ ]:
def train_using_entropy(X, y):
    # Decision tree with entropy
    dft_entropy = DecisionTreeClassifier(
        criterion="entropy", random_state=100,
        max_depth=3, min_samples_leaf=5)

    # Performing training
    dft_entropy.fit(X, y)
    return dft_entropy

In [ ]:
y_pred = train_using_entropy(X_train, Y_train).predict(X_test)
print("Accuracy : ", accuracy_score(Y_test, y_pred)*100)

In [ ]:
import xgboost as xgb
# define data_dmatrix
#https://gist.github.com/pb111/cc341409081dffa5e9eaf60d79562a03
data_dmatrix = xgb.DMatrix(data=X_train,label=Y_train)

In [ ]:
# import XGBClassifier
from xgboost import XGBClassifier


# declare parameters
params = {
            'objective':'binary:logistic',
            'max_depth': 4,
            'alpha': 10,
            'learning_rate': 1.0,
            'n_estimators':100
        }
            
            
            
# instantiate the classifier 
xgb_clf = XGBClassifier(**params)



# fit the classifier to the training data
xgb_clf.fit(X_train, Y_train)

In [ ]:
# make predictions on test data
y_pred2 = xgb_clf.predict(X_test)

In [ ]:

print('XGBoost model accuracy score: {0:0.4f}'. format(accuracy_score(Y_test, y_pred)))